# Similar Words

In [38]:
import pandas as pd
import jieba, re

csvfile = '/Users/xianghang/Desktop/sqlResult_1558435.csv'
content = pd.read_csv(csvfile, delimiter=",", quotechar='"', escapechar='\\')
content.fillna('')
news_content = content['content'].dropna().tolist()

In [39]:
def cut(s): return ' '.join(jieba.cut(s))
def token(s): return re.findall(r'[\d|\w]+', s)

In [41]:
import numpy as np
rawdata = [cut(' '.join(token(news))) for news in news_content]

rawdata[:2]

['此外   自 本周   6 月 12 日   起   除 小米 手机 6 等 15 款 机型 外   其余 机型 已 暂停 更新 发布   含 开发 版   体验版 内测   稳定版 暂不受 影响   以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作   有人 猜测 这 也 是 将 精力 主要 用到 MIUI   9 的 研发 之中   MIUI   8 去年 5 月 发布   距今已有 一年 有余   也 是 时候 更新换代 了   当然   关于 MIUI   9 的 确切 信息   我们 还是 等待 官方消息',
 '骁龙 835 作为 唯一 通过 Windows   10 桌面 平台 认证 的 ARM 处理器   高通 强调   不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心   相反   他们 正 联手 微软   找到 一种 适合 桌面 平台 的   兼顾 性能 和 功耗 的 完美 方案   报道 称   微软 已经 拿到 了 一些 新 的 源码   以便 Windows   10 更好 地 理解 big   little 架构   资料 显示   骁龙 835 作为 一款 集成 了 CPU   GPU   基带   蓝牙   Wi   Fi 的 SoC   比 传统 的 Wintel 方案 可以 节省 至少 30   的 PCB 空间   按计划   今年 Q4   华硕   惠普   联想 将 首发 骁龙 835   Win10 电脑   预计 均 是 二合一 形态 的 产品   当然   高通 骁龙 只是 个 开始   未来 也许 还 能 见到 三星 Exynos   联发科   华为 麒麟   小米 澎湃 等 进入 Windows   10 桌面 平台']

In [43]:
with open('news-sentences-cut.txt', 'w') as f:
    f.writelines(rawdata)

In [51]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [52]:
news_word2ve= Word2Vec(LineSentence('news-sentences-cut.txt'), size=35, workers=8)

In [72]:
news_word2ve.most_similar('表示')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('指出', 0.9100027084350586),
 ('说', 0.9051997065544128),
 ('认为', 0.8751715421676636),
 ('透露', 0.8278045058250427),
 ('强调', 0.7909775972366333),
 ('坦言', 0.7807946801185608),
 ('称', 0.7596263885498047),
 ('建议', 0.7534583806991577),
 ('介绍', 0.7453078627586365),
 ('看来', 0.7262009382247925)]

# More Data Better Result
1. 分词的问题
2. 数据量，量越多结果越准确

In [68]:
from collections import defaultdict

def getRelatedWords(iniwords, model):
    unseen = iniwords
    seen = defaultdict(int)
    max_size = 500
    
    while unseen and len(seen) < max_size:
        node = unseen.pop(0)
        new_expanding = [w for w,s in model.most_similar(node, topn=20)]
        unseen += new_expanding
        seen[node] += 1
        
    return seen

In [73]:
related_words = getRelatedWords(['说','表示'], news_word2ve)
sorted_related_words = sorted(related_words.items(), key=lambda x: x[1], reverse=True)
sorted_related_words[:20]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


[('表示', 83),
 ('说', 76),
 ('指出', 73),
 ('认为', 73),
 ('坦言', 61),
 ('透露', 60),
 ('所说', 51),
 ('提到', 49),
 ('称', 46),
 ('特别强调', 46),
 ('看来', 43),
 ('建议', 43),
 ('强调', 40),
 ('中说', 40),
 ('介绍', 35),
 ('明说', 34),
 ('文说', 34),
 ('告诉', 31),
 ('普遍认为', 25),
 ('时说', 24)]